### Word Segmenter 
##### ckiptagger: https://github.com/ckiplab/ckiptagger
(can use custom dictionary)
##### chinese Transformer: https://github.com/ckiplab/ckip-transformers
(CANNOT use custom dictionary, or at least I didn't find a way to do so)




速度：
jieba > ckip，但繁體中文精準度ckip勝
訓練sense embedding:可以先用coarse-grained訓，
將每個文章裡取出摘要區（page.summary）
對應意思的target加上suffix，
將加上suffix者列入斷詞custom dictionary，
pass進ckip裡斷詞。

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# get msw leveled list 
# SUBDIR = "/content/drive/MyDrive/110-1ComSem_final/SenseWdictv2/SenseTexts"
# with open (f'{DIR}/MSW_leveled_list.pkl', 'rb') as F:
#     leveled_list = pickle.load(F)
# read all texts out at once? (60K sentences)

Mounted at /content/drive


###**Installation and Init WS**

In [ ]:
!pip3 -q install ckiptagger
!pip3 -q install tensorflow
!pip3 -q install gdown

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ckiptagger import data_utils
# Sometimes the link inside data_utils is broken:
# default     file_id = "1efHsY16pxK0lBD2gYCgCTnv1Swstq771"
# alternative file_id = '16jCrS27EVCOFYWWx9Z0ByFG_Y-Df7CiJ'
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
100%|██████████| 1.88G/1.88G [00:15<00:00, 124MB/s]


In [ ]:
!pip3 -q install tensorflow-gpu

In [ ]:
# Set gpu env 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from ckiptagger import construct_dictionary, WS
# ws = WS("./data", disable_cuda=False)
ws = WS("./data", disable_cuda=False)

/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)


**test**

In [ ]:
word_to_weight = {
    "蘋果樹":1,
    "薔薇科蘋果亞科蘋果屬植物": 2,
    "礦物質": 1,
    "礦物質和維生素": 2
}
real_dictionary = construct_dictionary(word_to_weight)
print(real_dictionary)
sentlist = [
            "蘋果樹（學名：Malus domestica）是薔薇科蘋果亞科蘋果屬植物，為落葉喬木，在世界上廣泛種植。",
            "蘋果，又稱柰或林檎，是苹果樹的果實，一般呈紅色，但需視品種而定，富含礦物質和維生素，是人們最常食用的水果之一。",
            "人們根據需求的不同口感、用途（比如烹飪、生吃、釀蘋果酒等）培育不同的品種，已知有超過7,500個蘋果品種，擁有一系列人們需要的不同特性。"
            ]
word_sentence_list = ws(
    sentlist,
    sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    coerce_dictionary = real_dictionary, # words in this dictionary are forced
)
word_sentence_list

[(3, {'蘋果樹': 1.0, '礦物質': 1.0}), (7, {'礦物質和維生素': 2.0}), (12, {'薔薇科蘋果亞科蘋果屬植物': 2.0})]


[['蘋果樹',
  '（',
  '學名',
  '：',
  'Malus domestica',
  '）',
  '是',
  '薔薇科蘋果亞科蘋果屬植物',
  '，',
  '為',
  '落葉',
  '喬木',
  '，',
  '在',
  '世界',
  '上',
  '廣泛',
  '種植',
  '。'],
 ['蘋果',
  '，',
  '又',
  '稱',
  '柰',
  '或',
  '林檎',
  '，',
  '是',
  '苹果樹',
  '的',
  '果實',
  '，',
  '一般',
  '呈',
  '紅色',
  '，',
  '但',
  '需',
  '視',
  '品種',
  '而',
  '定',
  '，',
  '富含',
  '礦物質和維生素',
  '，',
  '是',
  '人們',
  '最',
  '常',
  '食用',
  '的',
  '水果',
  '之',
  '一',
  '。'],
 ['人們',
  '根據',
  '需求',
  '的',
  '不同',
  '口感',
  '、',
  '用途',
  '（',
  '比如',
  '烹飪',
  '、',
  '生吃',
  '、',
  '釀',
  '蘋果酒',
  '等',
  '）',
  '培育',
  '不同',
  '的',
  '品種',
  '，',
  '已',
  '知',
  '有',
  '超過',
  '7',
  ',',
  '500',
  '個',
  '蘋果',
  '品種',
  '，',
  '擁有',
  '一',
  '系列',
  '人們',
  '需要',
  '的',
  '不同',
  '特性',
  '。']]

### **load data for Word Segmenation**

In [ ]:
# https://drive.google.com/file/d/1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a/view?usp=sharing
# https://drive.google.com/file/d/1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a/view?usp=sharing
# download MSW.py
'''Initializaing data structure MSW'''
!gdown --id 1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a 

Downloading...
From: https://drive.google.com/uc?id=1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a
To: /content/MSW.py
100% 642/642 [00:00<00:00, 549kB/s]


In [ ]:
# load data 
import pickle  
W2VDIR = "/content/drive/MyDrive/110-1ComSem_final/w2v_resources"
with open (f'{W2VDIR}/MSW_leveled_list3.pkl', 'rb') as F:
    leveled_list = pickle.load(F)

In [ ]:
from tqdm import tqdm
import json
import re 
SEGDIR = "/content/drive/MyDrive/110-1ComSem_final/SenseWdictv2/SegTexts"
SUBDIR = '/content/drive/MyDrive/110-1ComSem_final/SenseWdictv2/SenseTexts'
for idx, msw in enumerate(tqdm(leveled_list)):
    word = msw.word
    print(f'\nSegmenting {word}...')
    # print(senses)
    coerce_dict = {}
    coerce_dict[word] = 2
    coerce_dict = construct_dictionary(coerce_dict)
    MSW_Seg_word= dict()
    for senseidx, sense in enumerate(msw.senses): 
          sensename = sense['sense']
          cat = sense['category']
          todel= set()
          filename = f'{cat}_{sensename}.txt'
          try:
            with open(SUBDIR+f"/{word}/{filename}", "r") as F:
                  text = F.read()
                  F.close()
                  if text.strip() == "":
                    todel.add(senseidx)
                    continue 
          except: todel.add(senseidx)
          # split to sentences
          text = re.sub(r"[!?？！]", '。', text)
          text_list = text.split('。')
          # print(text_list)
          print(f'-- Segmenting {cat}: {sensename}...')
          word_sent_list = ws(
              text_list,
              sentence_segmentation = True, # To consider delimiters
              segment_delimiter_set = {",", "。", ":", "?", "!", ";", "\n", "\t"}, # This is the defualt set of delimiters
              # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
              coerce_dictionary = coerce_dict, # words in this dictionary are forced
          )
          for j in range(0, len(word_sent_list), -1):
            word_sent_list[j] = [token.strip() for token in word_sent_list[j]]
            if len(word_sent_list[j]) <= 0: 
                del word_sent_list[j]
          MSW_Seg_word[f'{cat}_{sensename}'] = word_sent_list 
    print(len(MSW_Seg_word))
    with open (f'{SEGDIR}/MSW_Segdict_{word}.json', 'w') as fp:
        json.dump(MSW_Seg_word, fp, ensure_ascii=False, indent = 3)
    del MSW_Seg_word # release space in avoidance of crashes
    # update leveled_list
    todel = sorted(list(todel), reverse = True)
    print(len(leveled_list[idx].senses), todel)
    for todelidx in todel:
        del leveled_list[idx].senses[todelidx]
    # print(*MSW_Seg[word])

  0%|          | 0/9 [00:00<?, ?it/s]


Segmenting 洲...
-- Segmenting main: 洲...
-- Segmenting 香港: 大洲 (果洲羣島)...
-- Segmenting 香港: 大洲 (大浪灣)...
-- Segmenting 香港: 大洲 (西貢海)...
-- Segmenting 澳門: 大洲 (澳門)...
-- Segmenting 日本: 大洲市...
-- Segmenting 日本: 大洲城...
-- Segmenting 日本: 大洲藩...
-- Segmenting 中國大陸: 大洲鎮...
-- Segmenting 中國大陸: 大洲鄉...
-- Segmenting 中國大陸: 大洲島...
-- Segmenting 臺灣: 大洲 (宜蘭縣)...


 11%|█         | 1/9 [00:05<00:40,  5.03s/it]

12
12 []

Segmenting 紫羅蘭...
-- Segmenting main: 紫羅蘭...
-- Segmenting 其他: 紫羅蘭屬...
-- Segmenting 顏色: 紫羅蘭色...
-- Segmenting 人物: 紫羅蘭 (演員)...


 22%|██▏       | 2/9 [00:05<00:18,  2.57s/it]

...

-- Segmenting 其他: 黃海 (電影)...
13
13 []

Segmenting 黃河...
-- Segmenting main: 黃河...
-- Segmenting 河流: 黃河 (湄公河)...
-- Segmenting 河流: 黃河 (威斯康星河支流)...
-- Segmenting 地名: 黃河街道...
-- Segmenting 地名: 黃河鎮...
-- Segmenting 地名: 黃河鄉...
-- Segmenting 音樂: 黃河 (歌曲)...
-- Segmenting 音樂: 黃河大合唱...
-- Segmenting 音樂: 黃河鋼琴協奏曲...
-- Segmenting 人物: 黃河 (臺灣演員)...
-- Segmenting 人物: 黃河 (1968年)...
-- Segmenting 人物: 黃河 (華員會會長)...
-- Segmenting 人物: 黃河 (文藝工作者)...
-- Segmenting 人物: 黃河 (揚琴演奏家)...
-- Segmenting 人物: 黃河 (作家)...
-- Segmenting 人物: 黃河 (1960年)...
-- Segmenting 人物: 黃河 (1965年)...
-- Segmenting 人物: 黃河 (1967年)...
-- Segmenting 文學作品: 黃河 (雜誌)...


 89%|████████▉ | 8/9 [00:50<00:07,  7.58s/it]

19
19 [18]

Segmenting 黃油...
-- Segmenting main: 黃油...
-- Segmenting 其他: 黃油...


100%|██████████| 9/9 [00:51<00:00,  5.68s/it]

2
2 []


leveled_list (有Coarse的才會放上來)




```
├── msw object
│   ├── .word
│   ├── .idx
│   ├── .haslevel = True
│   ├── .senses
│   │   ├── sense1
│   │   │    ├── ["sense"]
│   │   │    ├── ["category"]
│   │   │    ├── ["views"]
│   │   │    ├── ["gloss"]
│   │   │    ├── ["link"]
│   │   │    ├── ["ismain"]
│   │   ├── sense2
│   │   ├── sense3 
...
```
JSON file 
SenseWdictv2/SegTexts/MSW_Segdict_{word}.json
```
├── category1_sense1
│   ├── sentence1(list)
│   │   │    ├── token
│   │   │    ├── token
│   │   │    ├── token
│   ├── sentence2
│   ├── sentence3
│   ├── sentence4
│   ├── sentence5
├── category1_sense2
├── ├── ...
├── category2_sense1
├── ├── ...
... 
...
```

In [ ]:
# UPDATE leveled_list
# with open (f'{DIR}/MSW_leveled_list2.pkl', 'wb') as F:
#     pickle.dump(leveled_list, F)
# with open (f'{SEGDIR}/MSW_leveled_list2.pkl', 'wb') as F:
#     pickle.dump(leveled_list, F)
W2VDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_resources'
with open (f'{W2VDIR}/MSW_leveled_list4.pkl', 'wb') as F:
    pickle.dump(leveled_list, F)